In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset
from datetime import datetime
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from timeit import default_timer as timer
import time

from tensorboardX import SummaryWriter

import math

In [2]:

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.src_tok_emb = nn.Linear(src_vocab_size, emb_size)

        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)


    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                memory_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         src_emb = self.positional_encoding(src)
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg).squeeze())
#         outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
#                                 src_padding_mask, tgt_padding_mask, memory_key_padding_mask)


        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask,memory_mask=memory_mask)
        return self.generator(outs)

#     def encode(self, src: Tensor, src_mask: Tensor):
#         return self.transformer.encoder(self.positional_encoding(
#                             self.src_tok_emb(src)), src_mask)

#     def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
#         return self.transformer.decoder(self.positional_encoding(
#                           self.tgt_tok_emb(tgt)), memory,
#                           tgt_mask)

In [3]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def generate_square_subsequent_mask2(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1)
    mask = mask.float().masked_fill(mask == 1, float('-inf')).masked_fill(mask == 0, float(0.0))
    return mask
def generate_square_subsequent_mask3(sz):
    mask = (torch.triu(torch.ones((sz, sz-1), device=DEVICE)) == 1).transpose(0,1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    src_mask = generate_square_subsequent_mask(src_seq_len)
    mmr_mask = generate_square_subsequent_mask3(src_seq_len)
    return src_mask, tgt_mask ,mmr_mask

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def batchify(data, bsz):
#     # 데이터셋을 bsz 파트들로 나눕니다.
#     nbatch = data.size(0) // bsz
#     # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
#     data = data.narrow(0, 0, nbatch * bsz)
#     # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
#     data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
#     return data.to(device)

def batchify(data, bsz,bptt):
    # 데이터셋을 bsz 파트들로 나눕니다.
    if(data.size(0)%(bsz*bptt)!=0):
        data=data.view(-1,bptt,data.size(1)).transpose(0,1).contiguous()
        return data.to(device)
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)


bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source)  - i)
    data = source[i:i+seq_len]
    if(seq_len!=bptt*bs):
        print(seq_len)
    target = source[i:i+seq_len].reshape(-1)
    return data, target

In [5]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    
    
    
    i=0
    while(i<Xtest_data.size(0)):
#     for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)

        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE,bptt)
        tgt=batchify(targets,BATCH_SIZE,bptt)
        src_input=src[:]
        tgt_input = tgt[:-1]


#         src_mask, tgt_mask = create_mask(src_input, tgt_input)
#         logits = model(src_input, tgt_input, src_mask, tgt_mask)
        src_mask, tgt_mask,mmr_mask = create_mask(src_input, tgt_input)
        logits = model(src_input, tgt_input, src_mask, tgt_mask,mmr_mask)
        
        optimizer.zero_grad()

        tgt_out = tgt[1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
        if(loss.isnan()):
            print('NNNNN')
            print(loss)
        _,predicted=torch.max(logits,-1)
        correct += (tgt_out.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt_out.squeeze()).sum().item()
        tot1+=(1== tgt_out.squeeze()).sum().item()
        tot2+=(2== tgt_out.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()
        
        conf00+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
        conf01+=((0== predicted)&(1==tgt_out.squeeze())).sum().item()
        conf02+=((0== predicted)&(2==tgt_out.squeeze())).sum().item()
        conf10+=((1== predicted) &(0==tgt_out.squeeze())).sum().item()
        conf11+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
        conf12+=((1== predicted)&(2==tgt_out.squeeze())).sum().item()
        conf20+=((2== predicted) &(0==tgt_out.squeeze())).sum().item()
        conf21+=((2== predicted)&(1==tgt_out.squeeze())).sum().item()
        conf22+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()

        
        
        
        i+=targets.size()[0]
    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
        
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc) 
    return losses / Xtrain_data.size(0),[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

In [6]:
def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    stime=time.time()
    with torch.no_grad():
        i=0
        while(i<Xtest_data.size(0)):
#         for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE,bptt)
            tgt=batchify(targets,BATCH_SIZE,bptt)

            src_input=src[:]
            
            tgt_input = tgt[:-1]
#             print(src_input.size())
#             print(tgt_input.size())
            src_mask, tgt_mask,mmr_mask = create_mask(src_input, tgt_input)

#             logits = model(src_input, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
            logits = model(src_input, tgt_input, src_mask, tgt_mask,mmr_mask)

            
            tgt_out = tgt[1:]

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            
            
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)
            
            correct += (tgt_out.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt_out.squeeze()).sum().item()
            tot1+=(1== tgt_out.squeeze()).sum().item()
            tot2+=(2== tgt_out.squeeze()).sum().item()

            conf00+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
            conf01+=((0== predicted)&(1==tgt_out.squeeze())).sum().item()
            conf02+=((0== predicted)&(2==tgt_out.squeeze())).sum().item()
            conf10+=((1== predicted) &(0==tgt_out.squeeze())).sum().item()
            conf11+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
            conf12+=((1== predicted)&(2==tgt_out.squeeze())).sum().item()
            conf20+=((2== predicted) &(0==tgt_out.squeeze())).sum().item()
            conf21+=((2== predicted)&(1==tgt_out.squeeze())).sum().item()
            conf22+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()
            

            
            i+=targets.size()[0]
    etime=time.time()
    print("Time elapsed",etime-stime)
    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
    
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc)
    return losses / Xtest_data.size(0),correct/total,prec,reca,f1sc,[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

In [7]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]

mbrnlist=mbrnlist1+mbrnlist2
mbrnlist=set(mbrnlist)
mbrnlist=list(mbrnlist)

# load array
MBR_NO,BRN_NO=mbrnlist[2]
featnorm=True

In [8]:

for MBR_NO,BRN_NO in mbrnlist:
#     if featnorm==True:
#         Data_train = load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
#         Data_test =  load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)
#     else:
#         Data_train = load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'train.npy',allow_pickle=True)
#         Data_test =  load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'test.npy',allow_pickle=True)
    
    if featnorm==True:
        Data_train = load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
        Data_test =  load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)
    else:
        Data_train = load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'train.npy',allow_pickle=True)
        Data_test =  load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'test.npy',allow_pickle=True)



    Xdata=[]
    Ydata=[]
    Xtrain_data=[]
    Ytrain_data=[]
    Xtest_data=[]
    Ytest_data=[]



    for idx in range(len(Data_train)//39):
        if(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any()):
            print(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any())
            continue
        Xtrain_data.append(Data_train[39*idx:39*(idx+1)][:,:].tolist())
        Ytrain_data.append(Data_train[39*idx:39*(idx+1)][:,-1].tolist())
    for idx in range(len(Data_test)//39):
        if(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any()):
            print(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any())
            continue
        Xtest_data.append(Data_test[39*idx:39*(idx+1)][:,:].tolist())
        Ytest_data.append(Data_test[39*idx:39*(idx+1)][:,-1].tolist())

    Xtrain_data=np.vstack(Xtrain_data)
    Ytrain_data=np.vstack(Ytrain_data)

    Xtrain_data=torch.FloatTensor(Xtrain_data)
    Ytrain_data=torch.IntTensor(Ytrain_data)
    Ytrain_data=Ytrain_data.view(-1)


    Xtest_data=np.vstack(Xtest_data)
    Ytest_data=np.vstack(Ytest_data)
    Xtest_data=torch.FloatTensor(Xtest_data)
    Ytest_data=torch.IntTensor(Ytest_data)
    Ytest_data=Ytest_data.view(-1)

    Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
    Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()
    Ytrain_data=Ytrain_data.T
    Ytest_data=Ytest_data.T

    torch.manual_seed(0)

    SRC_VOCAB_SIZE = Xtrain_data.shape[1]
    TGT_VOCAB_SIZE = 3
    EMB_SIZE =128
    NHEAD = 8
    FFN_HID_DIM = 128
    BATCH_SIZE = 32
    NUM_ENCODER_LAYERS = 3
    NUM_DECODER_LAYERS = 3

    transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                     NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
    summary = SummaryWriter()
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    transformer = transformer.to(DEVICE)

    loss_fn = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

    NUM_EPOCHS = 300
    best_val_loss=100000000
    for epoch in range(1, NUM_EPOCHS+1):
        start_time = timer()
        train_loss,_ = train_epoch(transformer, optimizer)
        end_time = timer()
        val_loss,acc,prec,reca,f1sc,confusion = evaluate(transformer)
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_confusion=confusion
            best_acc=acc
            best_prec=prec
            best_reca=reca
            best_f1sc=f1sc
            best_model = transformer
        summary.add_scalar('val loss', val_loss, epoch)
        summary.add_scalar('total loss', train_loss, epoch)
        summary.add_scalar('f1 score', f1sc, epoch)
        summary.add_scalar('Accuracy', acc, epoch)
    now = datetime.now()
    now.strftime("%m/%d/%Y, %H:%M:%S")

    date_time = now.strftime("%m_%d_%Y")

    PATH='best_model_Trans_seq_'+date_time+'_'+str(MBR_NO)+'_'+str(BRN_NO)
    if featnorm==True:
        torch.save(best_model.state_dict(), PATH+'norm')
    else:
        torch.save(best_model.state_dict(), PATH)
    if featnorm==True:
        file_name='result_Trans_'+date_time+'_norm.txt'
    else:
        file_name='result_Trans_'+date_time+'.txt'
    text_to_append=PATH+'\t'+"Acc:"+str(best_acc)+'\t'+"prec:"+str(best_prec)+'\t'+"recall:"+str(best_reca)+'\t'+"f1sc:"+str(best_f1sc)
    print(text_to_append)
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

8512
4134
Acc: 0.4856672932330827
Prec 0.37346717925817474
Recall 0.35703469796624066
F1 0.36506611606087097
312
312
Time elapsed 0.06718015670776367
8512
4719
Acc: 0.5543937969924813
Prec 0.40705333106293135
Recall 0.4178336469944828
F1 0.41237304591669566
Epoch: 1, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.668s
8512
5258
Acc: 0.6177161654135338
Prec 0.40428267974495197
Recall 0.43791353787344284
F1 0.4204266295299048
312
312
Time elapsed 0.05096578598022461
8512
4989
Acc: 0.5861137218045113
Prec 0.428891675298479
Recall 0.45626652082763647
F1 0.44215579397401195
Epoch: 2, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.234s
8512
5411
Acc: 0.6356907894736842
Prec 0.4153020372751315
Recall 0.44209727949994165
F1 0.4282809590768194
312
312
Time elapsed 0.040544986724853516
8512
5145
Acc: 0.6044407894736842
Prec 0.4407654595179322
Recall 0.48325201377606913
F1 0.4610319654576558
Epoch: 3, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.227s
8512
5466
Acc: 0.642152255639097

8512
5638
Acc: 0.662359022556391
Prec 0.5606835821215224
Recall 0.47916228370271763
F1 0.5167274006153117
312
312
Time elapsed 0.05566716194152832
8512
5207
Acc: 0.6117246240601504
Prec 0.5543571825957897
Recall 0.49279189143756114
F1 0.5217647254199377
Epoch: 27, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.242s
8512
5630
Acc: 0.6614191729323309
Prec 0.5265109395042435
Recall 0.4745810111683359
F1 0.49919908734305357
312
312
Time elapsed 0.03841280937194824
8512
5218
Acc: 0.613016917293233
Prec 0.5733097699167641
Recall 0.4964772338753418
F1 0.5321344299622797
Epoch: 28, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.237s
8512
5629
Acc: 0.6613016917293233
Prec 0.5357200854757711
Recall 0.47736157378091604
F1 0.5048599602452489
312
312
Time elapsed 0.04166102409362793
8512
5225
Acc: 0.6138392857142857
Prec 0.6014041474077166
Recall 0.4997307943367119
F1 0.5458734636562029
Epoch: 29, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.237s
8512
5650
Acc: 0.6637687969924813
Prec

8512
5680
Acc: 0.6672932330827067
Prec 0.5304678605277527
Recall 0.48217886330120296
F1 0.5051720091286176
312
312
Time elapsed 0.04144167900085449
8512
5225
Acc: 0.6138392857142857
Prec 0.5820314145602347
Recall 0.5006738017138207
F1 0.5382958845392393
Epoch: 53, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.246s
8512
5702
Acc: 0.6698778195488722
Prec 0.5436233592220443
Recall 0.48425294150453563
F1 0.5122235245384413
312
312
Time elapsed 0.050748586654663086
8512
5218
Acc: 0.613016917293233
Prec 0.5645026261887565
Recall 0.5004619767034785
F1 0.530556789191833
Epoch: 54, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.242s
8512
5705
Acc: 0.6702302631578947
Prec 0.5627528513762026
Recall 0.4865290738639904
F1 0.5218723719684939
312
312
Time elapsed 0.05646944046020508
8512
5229
Acc: 0.6143092105263158
Prec 0.5799358444700423
Recall 0.5036925445313377
F1 0.5391319831242881
Epoch: 55, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.235s
8512
5691
Acc: 0.6685855263157895
Prec 

8512
5731
Acc: 0.6732847744360902
Prec 0.575495724976521
Recall 0.4883289545657352
F1 0.528341241069453
312
312
Time elapsed 0.05090188980102539
8512
5236
Acc: 0.6151315789473685
Prec 0.5932370203676973
Recall 0.5001295388780568
F1 0.5427189169688171
Epoch: 79, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.267s
8512
5770
Acc: 0.6778665413533834
Prec 0.5640407369358664
Recall 0.49077743075266556
F1 0.5248648007643417
312
312
Time elapsed 0.05564713478088379
8512
5221
Acc: 0.6133693609022557
Prec 0.5574787614367241
Recall 0.4963255022553649
F1 0.5251277410804388
Epoch: 80, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.261s
8512
5736
Acc: 0.6738721804511278
Prec 0.5175028337654384
Recall 0.48201587208124685
F1 0.49912938749987984
312
312
Time elapsed 0.05395841598510742
8512
5235
Acc: 0.6150140977443609
Prec 0.5871252585372482
Recall 0.501239511465786
F1 0.5407936490954465
Epoch: 81, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.247s
8512
5739
Acc: 0.6742246240601504
Prec 0

8512
5762
Acc: 0.6769266917293233
Prec 0.5915515975734222
Recall 0.4901690813042953
F1 0.5361093834824082
312
312
Time elapsed 0.042394399642944336
8512
5228
Acc: 0.6141917293233082
Prec 0.5618006525815927
Recall 0.4999401911047254
F1 0.529068326389788
Epoch: 105, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.250s
8512
5760
Acc: 0.6766917293233082
Prec 0.5351874053489033
Recall 0.48784411651012854
F1 0.5104202976176921
312
312
Time elapsed 0.03820061683654785
8512
5236
Acc: 0.6151315789473685
Prec 0.5564394631134766
Recall 0.4995128927001126
F1 0.526441717378729
Epoch: 106, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.263s
8512
5780
Acc: 0.6790413533834586
Prec 0.5707452092432291
Recall 0.49150639685260994
F1 0.5281703877051493
312
312
Time elapsed 0.03923988342285156
8512
5245
Acc: 0.6161889097744361
Prec 0.5583920095147645
Recall 0.5033461821468319
F1 0.5294421700903394
Epoch: 107, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.230s
8512
5794
Acc: 0.6806860902255639
Pr

...

8512
7391
Acc: 0.8683035714285714
Prec 0.7483434080504602
Recall 0.6391277266573028
F1 0.6894370761046621
312
312
Time elapsed 0.052951812744140625
8512
6249
Acc: 0.7341400375939849
Prec 0.668177047513872
Recall 0.5999569871508844
F1 0.6322320470103492
Epoch: 183, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.275s
8512
7391
Acc: 0.8683035714285714
Prec 0.7490031237309233
Recall 0.6346632782248417
F1 0.6871089407617366
312
312
Time elapsed 0.05495810508728027
8512
6257
Acc: 0.7350798872180451
Prec 0.6723860534104067
Recall 0.6129270282278445
F1 0.6412812433425161
Epoch: 184, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.258s
8512
7397
Acc: 0.8690084586466166
Prec 0.7510250502175314
Recall 0.6423981211157318
F1 0.6924774771887321
312
312
Time elapsed 0.042089223861694336
8512
6263
Acc: 0.7357847744360902
Prec 0.676186693065587
Recall 0.6098785932814684
F1 0.6413232571323769
Epoch: 185, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.250s
8512
7385
Acc: 0.8675986842105263
Pre

8512
7406
Acc: 0.8700657894736842
Prec 0.7526478052885933
Recall 0.6565474844876272
F1 0.701320855742897
312
312
Time elapsed 0.038976192474365234
8512
6202
Acc: 0.7286184210526315
Prec 0.6505793521412876
Recall 0.5868624350656291
F1 0.6170804748122076
Epoch: 209, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.232s
8512
7400
Acc: 0.8693609022556391
Prec 0.7578378149835059
Recall 0.6319541800348728
F1 0.689194896335531
312
312
Time elapsed 0.0392765998840332
8512
6204
Acc: 0.7288533834586466
Prec 0.6567974394679295
Recall 0.5942678518218288
F1 0.6239699976526387
Epoch: 210, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.245s
8512
7392
Acc: 0.868421052631579
Prec 0.7531156399853544
Recall 0.638506418424793
F1 0.6910916179298155
312
312
Time elapsed 0.04136180877685547
8512
6193
Acc: 0.7275610902255639
Prec 0.6562587700256989
Recall 0.5926174210694862
F1 0.62281654918221
Epoch: 211, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.244s
8512
7395
Acc: 0.8687734962406015
Prec 0.75

8512
7440
Acc: 0.8740601503759399
Prec 0.7708358258706912
Recall 0.6558145937331685
F1 0.7086885154650974
312
312
Time elapsed 0.07056689262390137
8512
6164
Acc: 0.7241541353383458
Prec 0.6392601818773427
Recall 0.5887031758000089
F1 0.6129409267480196
Epoch: 235, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.233s
8512
7437
Acc: 0.8737077067669173
Prec 0.7698192576982713
Recall 0.6461833137343415
F1 0.7026037508006787
312
312
Time elapsed 0.0393524169921875
8512
6161
Acc: 0.7238016917293233
Prec 0.6345451945786208
Recall 0.5855744476107068
F1 0.6090770756426308
Epoch: 236, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.226s
8512
7448
Acc: 0.875
Prec 0.7726564772060683
Recall 0.6552287546183482
F1 0.7091140520595939
312
312
Time elapsed 0.06564784049987793
8512
6143
Acc: 0.721687030075188
Prec 0.6292560160603505
Recall 0.5874224345810506
F1 0.6076200342565116
Epoch: 237, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.257s
8512
7433
Acc: 0.8732377819548872
Prec 0.76413273956

8512
7457
Acc: 0.8760573308270677
Prec 0.7808545660722905
Recall 0.6657385614849661
F1 0.7187162521969861
312
312
Time elapsed 0.0673680305480957
8512
6133
Acc: 0.7205122180451128
Prec 0.6217667551457056
Recall 0.582936923218713
F1 0.6017260604639415
Epoch: 261, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.286s
8512
7448
Acc: 0.875
Prec 0.7609667737918793
Recall 0.6530497448602536
F1 0.7028901726630723
312
312
Time elapsed 0.04599404335021973
8512
6109
Acc: 0.7176926691729323
Prec 0.618355010464995
Recall 0.5809588178018844
F1 0.5990738827396812
Epoch: 262, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.230s
8512
7455
Acc: 0.8758223684210527
Prec 0.7664264993148745
Recall 0.6568785086762882
F1 0.7074366956531764
312
312
Time elapsed 0.03795933723449707
8512
6129
Acc: 0.7200422932330827
Prec 0.6232481571615941
Recall 0.5826949182879686
F1 0.6022896791791248
Epoch: 263, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.244s
8512
7472
Acc: 0.8778195488721805
Prec 0.767832590588

8512
7461
Acc: 0.8765272556390977
Prec 0.7920046974620227
Recall 0.6364193286833657
F1 0.7057387563453787
312
312
Time elapsed 0.04119753837585449
8512
6108
Acc: 0.7175751879699248
Prec 0.6206908014789215
Recall 0.5815938368724498
F1 0.6005066241860466
Epoch: 287, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.263s
8512
7464
Acc: 0.8768796992481203
Prec 0.7683391281942792
Recall 0.6849918473447502
F1 0.7242755403515035
312
312
Time elapsed 0.041330575942993164
8512
6140
Acc: 0.7213345864661654
Prec 0.6230838301071228
Recall 0.5727328849842865
F1 0.5968483214871816
Epoch: 288, Train loss: 0.000, Val loss: 0.000, Epoch time = 0.280s
8512
7488
Acc: 0.8796992481203008
Prec 0.7965500587972373
Recall 0.6635848245856905
F1 0.7240132909037058
312
312
Time elapsed 0.05912923812866211
8512
6134
Acc: 0.7206296992481203
Prec 0.624840418512029
Recall 0.5742762410454032
F1 0.5984922383261045
Epoch: 289, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.261s
8512
7490
Acc: 0.8799342105263158
Pre